https://gyro-interp.readthedocs.io/en/latest/examples.html

In [2]:
import numpy as np
from gyrointerp import gyro_age_posterior

# units: days
Prot, Prot_err = 11, 0.2

# units: kelvin
Teff, Teff_err = 4500, 100

# uniformly spaced grid between 0 and 2600 megayears
age_grid = np.linspace(0, 2600, 500)

# calculate the age posterior - takes ~30 seconds
age_posterior = gyro_age_posterior(
  Prot, Teff, Prot_err=Prot_err, Teff_err=Teff_err, age_grid=age_grid
)

# calculate dictionary of summary statistics
from gyrointerp import get_summary_statistics
result = get_summary_statistics(age_grid, age_posterior)

print(f"Age = {result['median']} +{result['+1sigma']} -{result['-1sigma']} Myr.")

Age = 571.01 +315.27 -174.57 Myr.


In [3]:
from gyrointerp.plotting import plot_prot_vs_teff

# write the results to the current working directory
outdir = "./"

# show these cluster Prot vs Teff datasets
reference_clusters = [
    'α Per', 'Pleiades', 'Blanco-1', 'Psc-Eri', 'NGC-3532', 'Group-X',
    'Praesepe', 'NGC-6811'
]

# underplot these polynomial fits
model_ids = [
    'α Per', '120-Myr', '300-Myr', 'Praesepe', 'NGC-6811'
]

# overplot these stars with big markers
custom_stardict = {
    "Kepler-1643": {"Prot":5.1, "Teff":4916, "m":"s", "c":"red"},
    "TOI-1136": {"Prot":8.7, "Teff":5770, "m":"X", "c":"pink"},
    "TOI-1937 A": {"Prot":6.6, "Teff":5798, "m":"P", "c":"aqua"},
}

# make the plot
plot_prot_vs_teff(
    outdir, reference_clusters=reference_clusters, model_ids=model_ids,
    custom_stardict=custom_stardict, writepdf=0
)

[W 240827 19:19:03 getters:72] No module named 'cdips'
[I 240827 19:19:03 getters:1551] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/alpha-per/Boyle_table3_full_DR3_supp.csv; returning.
[I 240827 19:19:03 getters:1253] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/psc-eri/Curtis_2019_X_GDR3_supplemented.csv, and not overwrite; returning.
[I 240827 19:19:03 getters:528] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/blanco1/Gillen_2020_X_GDR3_supplemented.csv, and not overwrite; returning.
[I 240827 19:19:03 getters:663] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/pleiades/Rebull_2016_X_GDR3_supplemented.csv, and not overwrite; returning.
[I 240827 19:19:03 getters:879] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-inter

2024-08-27T10:19:06.881348: made ./prot_vs_teff_α_Per_Pleiades_Blanco-1_Psc-Eri_NGC-3532_Group-X_Praesepe_NGC-6811_models_poly7_α_Per_120-Myr_300-Myr_Praesepe_NGC-6811_interpmethodpchip_m67.png
